<a href="https://colab.research.google.com/github/palswayam5/Hackathon/blob/main/Taxonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('/content/drive/MyDrive/hackathon/Doceree-HCP_Train.csv',encoding = "ISO-8859-1")

In [9]:
df.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113937 non-null  int64  
 1   DEVICETYPE       113937 non-null  object 
 2   PLATFORM_ID      113937 non-null  int64  
 3   BIDREQUESTIP     113937 non-null  object 
 4   USERPLATFORMUID  113933 non-null  object 
 5   USERCITY         107578 non-null  object 
 6   USERZIPCODE      109345 non-null  float64
 7   USERAGENT        113935 non-null  object 
 8   PLATFORMTYPE     113937 non-null  object 
 9   CHANNELTYPE      113937 non-null  object 
 10  URL              113937 non-null  object 
 11  KEYWORDS         113937 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(10)
memory usage: 12.2+ MB


In [4]:
ohe = pd.get_dummies(df,columns = ['DEVICETYPE','PLATFORM_ID','PLATFORMTYPE'])

In [5]:
drop = ohe.drop(['USERPLATFORMUID','ID','USERCITY','USERAGENT','CHANNELTYPE','URL'],axis = 1)

In [6]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split('|'))
vectorizer = TfidfVectorizer()

In [7]:
bid = vectorizer.fit_transform(df['BIDREQUESTIP'])
bid = pd.DataFrame(bid.toarray())

In [15]:
bid.head()

,0,1,2,3,4,5,6,7,8,9,...,236,237,238,239,240,241,242,243,244,245
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.48034


In [8]:
kywrds = tfidf.fit_transform(df['KEYWORDS'])
kywrds = pd.DataFrame(kywrds.toarray(),columns = tfidf.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
kywrds.head()

,,abatacept,ablation techniques,abortion,abscess,acanthamoeba keratitis,acetaminophen,acid-base imbalance,aco,acos,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
drop = drop.drop(['BIDREQUESTIP','KEYWORDS'],axis = 1)

In [19]:
drop.head()

,USERZIPCODE,TAXONOMY,IS_HCP,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,...,PLATFORM_ID_12,PLATFORM_ID_13,PLATFORM_ID_14,PLATFORM_ID_15,PLATFORM_ID_16,PLATFORMTYPE_Electronic Health Record,PLATFORMTYPE_Medical News Platform,PLATFORMTYPE_Online Learning Portal,PLATFORMTYPE_Online Medical Journal,PLATFORMTYPE_Tele-medicine Platform
0,97206.0,NaN,0.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,22202.0,NaN,0.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,83654.0,NaN,0.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,229114624.0,2084P0800X,1.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,77008.0,NaN,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
fa = pd.concat([drop,bid,kywrds],axis = 1)

In [11]:
del kywrds
del bid

In [12]:
fa.drop('IS_HCP',axis = 1,inplace = True)

In [23]:
fa.head()

,USERZIPCODE,TAXONOMY,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,PLATFORM_ID_4,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,97206.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22202.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83654.0,NaN,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,229114624.0,2084P0800X,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,77008.0,NaN,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
fa = fa.dropna(how = 'any',axis = 0)
null = dict(fa.isnull().sum())

since no output so there are no null values

In [25]:
fa.info() #object is taxonomy

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32313 entries, 3 to 113936
Columns: 1488 entries, USERZIPCODE to zoster sine herpete
dtypes: float64(1463), object(1), uint8(24)
memory usage: 361.9+ MB


In [14]:
le = LabelEncoder()

In [15]:
le.fit(fa['TAXONOMY'])

LabelEncoder()

In [16]:
classes = le.classes_

In [17]:
y = le.transform(fa['TAXONOMY'])

In [18]:
y = y.reshape(-1,1)
y.shape

(32313, 1)

In [19]:
print(fa.shape,y.shape)

(32313, 1488) (32313, 1)


no null in y values as well so we are ready to train

In [20]:
X = fa.drop(['TAXONOMY'],axis = 1)

In [21]:
col = list(X.columns)

In [34]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32313 entries, 3 to 113936
Columns: 1487 entries, USERZIPCODE to zoster sine herpete
dtypes: float64(1463), uint8(24)
memory usage: 361.7 MB


In [22]:
sc = StandardScaler()

In [23]:
X.columns = X.columns.astype(str)

In [24]:
X = sc.fit_transform(X)

pca and trainig takes too much time not gonna do these here just gonna show you the results

In [25]:
pca = PCA()
X = pca.fit_transform(X)

In [26]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.4)

In [27]:
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
hclf = HistGradientBoostingClassifier()
xgb = GradientBoostingClassifier()

In [28]:
rf.fit(X_train,y_train)

<ipython-input-28-d6d9475b85dc>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)


RandomForestClassifier()

In [42]:
from sklearn.model_selection import cross_val_score

In [43]:
scores = cross_val_score(rf,X,y,cv = 5)
scores.mean()

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel

0.6931264475118779

In [44]:
y_pred_val = rf.predict(X_val)

In [45]:
y_pred_val

array([143, 149, 156, ..., 149, 149,  60])

In [46]:
y_val

array([[ 18],
       [ 66],
       [156],
       ...,
       [143],
       [112],
       [ 33]])

In [47]:
accuracy_score(y_val,y_pred_val)

0.6490020114497911

In [29]:
df_test = pd.read_csv('/content/drive/MyDrive/hackathon/Doceree-HCP_Test.csv')

In [30]:
df_test = pd.get_dummies(df_test,columns = ['DEVICETYPE','PLATFORM_ID','PLATFORMTYPE'])
df_test = df_test.drop(['USERPLATFORMUID','ID','USERCITY','USERAGENT','CHANNELTYPE','URL'],axis = 1)

In [31]:
bid = vectorizer.transform(df_test['BIDREQUESTIP'])
bid = pd.DataFrame(bid.toarray())

In [32]:
kywrds = tfidf.transform(df_test['KEYWORDS'])
kywrds = pd.DataFrame(kywrds.toarray(),columns = tfidf.get_feature_names_out())

In [33]:
df_test = df_test.drop(['BIDREQUESTIP','KEYWORDS'],axis = 1)

In [34]:
df_test = pd.concat([df_test,bid,kywrds],axis = 1)

In [35]:
del kywrds
del bid

In [36]:
df_test.shape

(28493, 1485)

In [37]:
trcol = list(fa.columns)
tstcol = list(df_test.columns)

In [38]:
msngcol = []
index = []
ind = 0
for i in trcol:
  if i not in tstcol:
    msngcol.append(i)
    index.append(ind)
  ind+=1
index

[1, 20, 25]

In [39]:
msngcol

['TAXONOMY', 'PLATFORM_ID_16', 'PLATFORMTYPE_Tele-medicine Platform']

inculcating missing columns

In [40]:
pid16 = np.zeros([df_test.shape[0],1])
tele = np.zeros([df_test.shape[0],1])
df_test.insert(19, "PLATFORM_ID_16",pid16)
df_test.insert(24, "PLATFORMTYPE_Tele-medicine Platform",tele)

In [60]:
df_test.head()

,USERZIPCODE,DEVICETYPE_Desktop,DEVICETYPE_Mobile,DEVICETYPE_Tablet,DEVICETYPE_Unknown,PLATFORM_ID_1,PLATFORM_ID_2,PLATFORM_ID_3,PLATFORM_ID_4,PLATFORM_ID_5,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,28305.0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16316.0,0,1,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,91724.0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11226.0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30062.0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_test["USERZIPCODE"].fillna( method ='ffill', inplace = True)

In [62]:
df_test.isnull().sum()

USERZIPCODE            0
DEVICETYPE_Desktop     0
DEVICETYPE_Mobile      0
DEVICETYPE_Tablet      0
DEVICETYPE_Unknown     0
                      ..
yaws                   0
yellow fever           0
zoonoses               0
zostavax               0
zoster sine herpete    0
Length: 1487, dtype: int64

In [42]:
col_t = list(df_test.columns)

In [43]:
print(col_t[20],"  ",col[20])

PLATFORMTYPE_Electronic Health Record    PLATFORMTYPE_Electronic Health Record


In [44]:
X_test = df_test
X_test.columns = X_test.columns.astype(str)
X_test = sc.transform(X_test)

In [45]:
y_test = rf.predict(X_test)

In [48]:
y_test

array([149, 149, 149, ..., 149, 149, 149])

In [47]:
hcp = pd.read_csv('/content/IS_HCP5.csv')

In [49]:
ind = []
for i in range(hcp.shape[0]):
  if(hcp['IS_HCP'][i]==1):
    ind.append(i)

In [50]:
for i in range(y_test.shape[0]):
  if i not in ind:
    y_test[i] = 0
y_test

array([  0, 149,   0, ...,   0,   0,   0])

In [51]:
fa = []
for i in range(y_test.shape[0]):
  if(y_test[i]==0):
    fa.append('')
  else:
    fa.append(classes[y_test[i]])

In [52]:
fa = pd.DataFrame(fa,columns = ['TAXONOMY'])

In [53]:
fa.head()

,TAXONOMY
0,
1,2084P0800X
2,
3,
4,


In [54]:
fa = pd.concat([hcp,fa],axis = 1)
fa.head()

,ID,IS_HCP,TAXONOMY
0,115501,0,
1,115502,1,2084P0800X
2,115503,0,
3,115504,0,
4,115505,0,


final result is sotred in TAXONOMY.csv not gonna run the last tab coz results are already stored

In [55]:
fa.to_csv('TAXONOMY2.csv')